# Import

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.6 MB/s eta 0:00:00


In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 81 (delta 4), reused 4 (delta 4), pack-reused 67 (from 1)
Receiving objects: 100% (81/81), 105.83 MiB | 39.25 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
%cd hackaton/

/kaggle/working/hackaton


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1TJC31Ksuxbfk1pHnKPrPQWLbQIn0aTfi -O datasets

Retrieving folder contents
Retrieving folder 1iWOA5sQM79qMSQLhZMOKmpEGUG3Y0fZR A
Processing file 1pNAwAj6YXp-AhpyvNCq4Ht5Oxl44FyHI test.json.gz
Processing file 1LAQd0cPGi9lbEkeDE1Kij_t8XwJZYP8C train.json.gz
Retrieving folder 1H3dxcEl6688pL6oz3QDIR8KdYwNalucU B
Processing file 1mMu3gB-080GrIqhlb-Uq_uasESRlChEo test.json.gz
Processing file 1mWWX8-PH0S2AmZG5mjzvBxn11JhG3mtp train.json.gz
Retrieving folder 1HVROb8adsGMBkFQvKbQUkh3ZrnjNQjJJ C
Processing file 1UTMz6ATbZwg47RWShC0UEoxrh6wbSCHT test.json.gz
Processing file 13gamiak1mUzcEcM90PNq8RwZFwsLUkK4 train.json.gz
Retrieving folder 1BQ871PttFl5GRpioBsAMgfVqdoYwRiEG D
Processing file 1JSIlf6MFDo2HZ3zGmjRWYBG6H81sggte test.json.gz
Processing file 1W9OmdHD-ZeRdfmzEFsdO_YWkiLeQ9D-b train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1pNAwAj6YXp-AhpyvNCq4Ht5Oxl44FyHI
From (redirected): https://drive.google.com/uc?

In [5]:
!ls -lh datasets

total 16K
drwxr-xr-x 2 root root 4.0K Jun  1 11:58 A
drwxr-xr-x 2 root root 4.0K Jun  1 11:58 B
drwxr-xr-x 2 root root 4.0K Jun  1 11:59 C
drwxr-xr-x 2 root root 4.0K Jun  1 11:59 D


In [6]:
import os
import torch
import torch.nn as nn
from torch.nn import LazyLinear
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse
from torch_geometric.nn import (
    GINEConv, JumpingKnowledge, global_mean_pool, global_add_pool, global_max_pool,
    Set2Set, GlobalAttention
)

# Set the random seed
set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Utils

In [7]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [8]:
def train(
    data_loader,
    model,
    optimizer,
    scheduler,
    criterion,
    device,
    per_step_scheduler=False,
    save_checkpoints=False,
    checkpoint_path="ckpt",
    current_epoch=0,
    grad_clip: float = 1.0,
):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for data in tqdm(data_loader, desc="Train-batches", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss   = criterion(output, data.y)
        loss.backward()

        # Gradient-clipping per stabilità
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total   += data.y.size(0)
        if per_step_scheduler:  # OneCycleLR vuole step() per batch
            scheduler.step()
        optimizer.zero_grad()
    
    if not per_step_scheduler:  # CosineAnnealing, ecc.
        scheduler.step()

    # Eventuale checkpoint
    if save_checkpoints:
        fname = f"{checkpoint_path}_epoch_{current_epoch+1}.pth"
        torch.save(model.state_dict(), fname)
        print(f"✓ Checkpoint salvato: {fname}")

    return total_loss / len(data_loader), correct / total

In [9]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [10]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "my_submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [11]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [12]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [71]:
def get_arguments():
    args = {}
    args['train_path'] = "/kaggle/working/hackaton/datasets/B/train.json.gz"
    args['test_path'] = "/kaggle/working/hackaton/datasets/B/test.json.gz"
    args['num_checkpoints'] = 5
    args['drop_ratio'] = 0.1
    args['batch_size'] = 32
    args['epochs'] = 100
    return argparse.Namespace(**args)


In [72]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Arguments received:
train_path: /kaggle/working/hackaton/datasets/B/train.json.gz
test_path: /kaggle/working/hackaton/datasets/B/test.json.gz
num_checkpoints: 5
drop_ratio: 0.1
batch_size: 32
epochs: 100


In [32]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SymmetricCrossEntropyLoss(nn.Module):
    """
    SCE = α · CE + β · RCE  (Ghosh et al., 2021)
    • CE  = Cross-Entropy classica                        (robusta < α)
    • RCE = Reversed Cross-Entropy (penalizza outlier)    (robusta < β)

    Args
    ----
    alpha        peso della CE         (tipico 0.1 – 1.0)
    beta         peso della RCE        (tipico   1  – 10 )
    num_classes  # classi del dataset
    epsilon      smoothing per evitare log(0) nel calcolo di RCE
    """
    def __init__(self,
                 alpha: float = 1.0,
                 beta: float = 1.0,
                 num_classes: int = 6,
                 epsilon: float = 1e-4):
        super().__init__()
        self.alpha = alpha
        self.beta  = beta
        self.eps   = epsilon
        self.num_classes = num_classes
        self.ce = nn.CrossEntropyLoss()            # CE con reduction='mean'

    # ----------------------------------------------------------------- forward
    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # --- CE classica -----------------------------------------------------
        ce_loss = self.ce(logits, targets)

        # --- RCE (reversed CE) ----------------------------------------------
        # p = modello (softmax);   ỹ = one-hot label (add smoothing)
        p = F.softmax(logits, dim=1)                               # [B, C]
        y_tilde = torch.zeros_like(p).scatter_(1, targets.unsqueeze(1), 1)
        y_tilde = y_tilde * (1 - self.eps) + self.eps / self.num_classes

        rce_loss = (- (p * torch.log(y_tilde)).sum(dim=1)).mean()

        # combinazione pesata
        return self.alpha * ce_loss + self.beta * rce_loss


In [55]:
class EarlyStop:
    def __init__(self, patience=10):
        self.patience  = patience
        self.best_val  = None
        self.counter   = 0

    def step(self, current_val):
        if self.best_val is None or current_val > self.best_val:
            self.best_val = current_val
            self.counter  = 0
            return False      # continua il training
        self.counter += 1
        return self.counter >= self.patience

# Model

In [56]:
class GINE_GNN(torch.nn.Module):
    """
    GNN per graph-classification con GINEConv.
    Usa edge-attributes (edge_dim=7) e gestisce label-noise con Dropout profondo
    + JumpingKnowledge.
    """

    def __init__(
        self,
        num_class: int,
        num_layers: int = 5,
        emb_dim: int = 128,
        edge_dim: int = 7,
        jk_mode: str = "last",
        graph_pooling: str = "mean",
        dropout: float = 0.2,
    ):
        super().__init__()

        self.num_layers, self.jk_mode, self.dropout = num_layers, jk_mode, dropout

        # embedding “fittizia” per i nodi: tutti gli x=0 →  embedding di un solo indice
        self.node_emb = nn.Embedding(1, emb_dim)

        # GINE ⟶ usa edge_attr
        self.convs = nn.ModuleList()
        self.bns   = nn.ModuleList()
        for i in range(num_layers):
            mlp = nn.Sequential(
                nn.Linear(emb_dim, emb_dim),
                nn.BatchNorm1d(emb_dim),
                nn.ReLU(),
                nn.Linear(emb_dim, emb_dim),
            )
            self.convs.append(GINEConv(nn=mlp, edge_dim=edge_dim))
            self.bns.append(nn.BatchNorm1d(emb_dim))

        # JumpingKnowledge opzionale
        if jk_mode in {"cat", "max", "lstm"}:
            self.jump = JumpingKnowledge(jk_mode, channels=emb_dim, num_layers=num_layers)
            out_dim = emb_dim * (num_layers if jk_mode == "cat" else 1)
        else:                              # "last"  ↦ usa solo l’ultimo layer
            self.jump, out_dim = None, emb_dim

        # pooling a livello di grafo
        if graph_pooling == "sum":
            self.pool = global_add_pool
        elif graph_pooling == "max":
            self.pool = global_max_pool
        elif graph_pooling == "attention":
            self.pool = GlobalAttention(
                gate_nn=nn.Sequential(
                    nn.Linear(out_dim, out_dim * 2),
                    nn.BatchNorm1d(out_dim * 2),
                    nn.ReLU(),
                    nn.Linear(out_dim * 2, 1),
                )
            )
        elif graph_pooling == "set2set":
            self.pool = Set2Set(out_dim, processing_steps=2)
            out_dim *= 2
        else:                              # default “mean”
            self.pool = global_mean_pool

        self.head = nn.Sequential(
            nn.Linear(out_dim, out_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(out_dim, num_class),
        )

    # --------------------------------------------------------------------- forward
    def forward(self, data):
        x   = self.node_emb(torch.zeros(data.num_nodes,
                                        dtype=torch.long,
                                        device=data.edge_index.device))
        xs  = []

        for conv, bn in zip(self.convs, self.bns):
            x = conv(x, data.edge_index, data.edge_attr)
            x = bn(x).relu()
            x = nn.functional.dropout(x, p=self.dropout, training=self.training)
            xs.append(x)

        x = self.jump(xs) if self.jump else x
        g = self.pool(x, data.batch)
        return self.head(g)

In [73]:
model = GINE_GNN(
    num_class=6,
    num_layers=5,
    emb_dim=256,
    edge_dim=7,
    jk_mode="max",
    graph_pooling="global_attention",# "global_attention"
    dropout=0.3,
).to(device)

# Other

In [74]:
script_dir = os.getcwd() 
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [75]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [76]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)

In [77]:
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
optimizer = torch.optim.AdamW(
    model.parameters(), lr=3e-3, weight_decay=0.01
)

# A, C, D NoisyLoss, B invece SCE
#criterion  = NoisyCrossEntropyLoss(0.2).to(device)
criterion = SymmetricCrossEntropyLoss(
                alpha=0.1,
                beta=4.0,
                num_classes=6
           ).to(device)

In [ ]:
if args.train_path:
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size
    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-3,         # picco
    steps_per_epoch=len(train_loader),
    epochs=args.epochs,
    div_factor=25,       # warm-up
    final_div_factor=1e3 # annealing verso 3e-6
)
    
    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    early_stop = EarlyStop(patience=20)
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, scheduler, criterion, device,
            per_step_scheduler = True,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        if early_stop.step(val_acc):
            print(f"Early stopping a epoch {epoch+1}")
            break

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs: 100%|██████████| 35/35 [00:04<00:00,  7.93batch/s]
Epoch 1/100, Loss: 34.2508, Train Acc: 0.2654, Val Acc: 0.2607
Epoch 1/100, Loss: 34.2508, Train Acc: 0.2654, Val Acc: 0.2607
Epoch 1/100, Loss: 34.2508, Train Acc: 0.2654, Val Acc: 0.2607
Epoch 1/100, Loss: 34.2508, Train Acc: 0.2654, Val Acc: 0.2607


Epoch 1/100, Loss: 34.2508, Train Acc: 0.2654, Val Acc: 0.2607
Best model updated and saved at /kaggle/working/hackaton/checkpoints/model_B_best.pth


Iterating eval graphs: 100%|██████████| 35/35 [00:04<00:00,  8.02batch/s]
Epoch 2/100, Loss: 31.5891, Train Acc: 0.3013, Val Acc: 0.2589
Epoch 2/100, Loss: 31.5891, Train Acc: 0.3013, Val Acc: 0.2589
Epoch 2/100, Loss: 31.5891, Train Acc: 0.3013, Val Acc: 0.2589
Epoch 2/100, Loss: 31.5891, Train Acc: 0.3013, Val Acc: 0.2589


Epoch 2/100, Loss: 31.5891, Train Acc: 0.3013, Val Acc: 0.2589


Iterating eval graphs: 100%|██████████| 35/35 [00:04<00:00,  7.89batch/s]
Epoch 3/100, Loss: 30.7781, Train Acc: 0.3134, Val Acc: 0.3143
Epoch 3/100, Loss: 30.7781, Train Acc: 0.3134, Val Acc: 0.3143
Epoch 3/100, Loss: 30.7781, Train Acc: 0.3134, Val Acc: 0.3143
Epoch 3/100, Loss: 30.7781, Train Acc: 0.3134, Val Acc: 0.3143


...

Iterating eval graphs: 100%|██████████| 35/35 [00:04<00:00,  7.75batch/s]
Epoch 70/100, Loss: 19.0481, Train Acc: 0.5815, Val Acc: 0.6000
Epoch 70/100, Loss: 19.0481, Train Acc: 0.5815, Val Acc: 0.6000
Epoch 70/100, Loss: 19.0481, Train Acc: 0.5815, Val Acc: 0.6000
Epoch 70/100, Loss: 19.0481, Train Acc: 0.5815, Val Acc: 0.6000


Epoch 70/100, Loss: 19.0481, Train Acc: 0.5815, Val Acc: 0.6000
Best model updated and saved at /kaggle/working/hackaton/checkpoints/model_B_best.pth


Iterating eval graphs: 100%|██████████| 35/35 [00:04<00:00,  7.83batch/s]
Epoch 71/100, Loss: 18.9716, Train Acc: 0.5833, Val Acc: 0.5661
Epoch 71/100, Loss: 18.9716, Train Acc: 0.5833, Val Acc: 0.5661
Epoch 71/100, Loss: 18.9716, Train Acc: 0.5833, Val Acc: 0.5661
Epoch 71/100, Loss: 18.9716, Train Acc: 0.5833, Val Acc: 0.5661


Epoch 71/100, Loss: 18.9716, Train Acc: 0.5833, Val Acc: 0.5661


Train-batches:  41%|████      | 57/140 [00:09<00:13,  6.06batch/s]

In [ ]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

In [ ]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

In [ ]:
del test_dataset
del test_loader

In [ ]:
!zip -r output.zip /kaggle/working/hackaton